# 調和励振系の正規形（数値シミュレーション）

吉田勝俊（宇都宮大学）

## 参考情報

- [【Python】行列指数関数・行列対数関数 - Qiita](https://qiita.com/Mrrmm252/items/a50a9b352e5064e40cc1)
- [[Python] Numpyの参照、抽出、結合 - Qiita](https://qiita.com/supersaiakujin/items/d63c73bb7b5aac43898a)
<!-- - [Pythonで運動方程式を解く(odeint) - Qiita](https://qiita.com/binaryneutronstar/items/ad5efa27fd626826846f) -->
<!-- - [2つの信号間の遅延を推定する - Qiita](https://qiita.com/inoory/items/3ea2d447f6f1e8c40ffa) -->

In [8]:
%matplotlib inline

import numpy as np                 #数値計算ライブラリ
from scipy.linalg import expm      #行列指数関数
import matplotlib.pyplot as plt    #描画ライブラリ
plt_config = {
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 14,
    'lines.linewidth': 1.5,
    'text.usetex': True,
}
plt.rcParams.update(plt_config)

## ◯行列指数関数

### ■指数関数

In [2]:
np.exp(2) #普通の指数関数

7.38905609893065

### ■行列指数関数

#### お試し用の行列

In [19]:
A = np.array([
    [0, 1],
    [-2, -3]
])
display(A)

array([[ 0,  1],
       [-2, -3]])

#### （誤）Numpy の `exp` $\neq$ 行列指数関数

In [20]:
np.exp(A) #これは単なる各成分の指数関数値

array([[1.        , 2.71828183],
       [0.13533528, 0.04978707]])

In [21]:
np.exp(A[0,0]), np.exp(A[0,1]), np.exp(A[1,0]), np.exp(A[1,1]) 

(1.0, 2.718281828459045, 0.1353352832366127, 0.049787068367863944)

#### （正）<font color="red">Scipy の `expm` $=$ 行列指数関数</font>．

In [22]:
expm(A)

array([[ 0.6004236 ,  0.23254416],
       [-0.46508832, -0.09720887]])